In [1]:
import pandas as pd
import numpy as np
from numpy import array
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [12]:
data = np.array(pd.read_csv('../Refined Dataset.csv'))[:, -1]
split = int(0.1*data.shape[0])

In [15]:
data = data[:split]
data.shape

(23821,)

In [27]:
data

array(["Oh baby, how you doing?\nYou know I'm gonna cut right to the chase\nSome women were made but me, myself\nI like to think that I was created for a special purpose\nYou know, what's more special than you? You feel me\nIt's on baby, let's get lost\nYou don't need to call into work 'cause you're the boss\nFor real, want you to show me how you feel\nI consider myself lucky, that's a big deal\nWhy? Well, you got the key to my heart\nBut you ain't gonna need it, I'd rather you open up my body\nAnd show me secrets, you didn't know was inside\nNo need for me to lie\nIt's too big, it's too wide\nIt's too strong, it won't fit\nIt's too much, it's too tough\nHe talk like this 'cause he can back it up\nHe got a big ego, such a huge ego\nI love his big ego, it's too much\nHe walk like this 'cause he can back it up\nUsually I'm humble, right now I don't choose\nYou can leave with me or you could have the blues\nSome call it arrogant, I call it confident\nYou decide when you find on what I'm w

In [69]:
lines = ""
for ix in range(data.shape[0]):
    cur = data[ix].split('\n')
    for jx in range(len(cur)):
        #lines.append(cur[jx])
        lines += cur[jx]
        lines += " "
len(lines)

28051993

In [65]:
print(lines[:100])

Oh baby, how you doing? You know I'm gonna cut right to the chase Some women were made but me, mysel


In [66]:
import re

In [89]:
def clean_doc(doc):
    letters_only = re.sub("[^a-zA-Z' ]", "", doc)
    letters_only = letters_only.lower()
    tokens = letters_only.split()
    return tokens

clean = clean_doc(lines)

In [90]:
clean[:10]

['oh', 'baby', 'how', 'you', 'doing', 'you', 'know', "i'm", 'gonna', 'cut']

In [91]:
freq = {}
for ix in range(len(clean)):
    try:
        freq[clean[ix]] += 1
    except:
        freq[clean[ix]] = 1

In [93]:
new_list = []
for wrd, f in freq.items():
    if f > 10:
        new_list.append(wrd)
print(len(new_list))
print(len(clean))

12884
5573084


In [ ]:
with open('your_file.txt', 'w') as f:
    for item in lines:
        f.write("%s\n" % item)

In [ ]:
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1

# separate into input and output
sequences = array(sequences)
sequences = pad_sequences(sequences, maxlen = 10, padding='post')
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

In [ ]:
# define model
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

In [ ]:
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X, y, batch_size=128, epochs=10)

# save the model to file
model.save('model.h5')
# save the tokenizer
dump(tokenizer, open('tokenizer.pkl', 'wb'))

In [ ]:
from random import randint
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences

# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# generate a sequence from a language model
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
	result = list()
	in_text = seed_text
	# generate a fixed number of words
	for _ in range(n_words):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
		# predict probabilities for each word
		yhat = model.predict_classes(encoded, verbose=0)
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text += ' ' + out_word
		result.append(out_word)
	return ' '.join(result)

# load cleaned text sequences
#in_filename = 'republic_sequences.txt'
#doc = load_doc(in_filename)
#lines = doc.split('\n')
seq_length = len(lines[0].split()) - 1

# load the model
model = load_model('model.h5')

# load the tokenizer
tokenizer = load(open('tokenizer.pkl', 'rb'))

# select a seed text
seed_text = lines[randint(0,len(lines))]
print(seed_text + '\n')

# generate new text
generated = generate_seq(model, tokenizer, seq_length, seed_text, 50)
print(generated)

In [26]:
mf = []
words = lines.split()
cnt = 0
for wx in words:
    wx = wx.lower()
    try:
        if freq[wx] > 10:
            mf.append(wx)
    except :
        cnt += 1
print cnt

17005


In [27]:
print mf[:10]

['oh', 'baby', 'how', 'you', 'doing', 'you', 'know', 'im', 'gonna', 'cut']


In [ ]:
# define model
vocab_size = len(mf)
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

In [29]:
with open('data.txt', 'w') as f:
    for item in mf:
        f.write("%s " % item)